# Prep OPERA RTC CalVal Slope Comparison: part 1

**Alex Lewandowski; Alaska Satellite Facility, University of Alaska Fairbanks**

## Performs initial data preparation for the OPERA RTC CalVal Slope Comparison Module

**Notebook Requires**
- dual-pol OPERA RTC samples
- local incidence angle map
- ellipsoidal incidence angle map
- layover-shadow mask

**Actions**
1. identifies and downloads required [Copernicus Global Land Cover (100m)](https://lcviewer.vito.be/download) data
1. mosaics land cover data
1. creates shapefiles from VH RTC
1. subsets all geotiffs with shapefile

In [ ]:
from ipyfilechooser import FileChooser
import numpy as np
from pathlib import Path
from glob import glob
import os
import shutil
import sys
from tqdm.auto import tqdm

import branca
import branca.colormap as cm
from branca.element import Template, MacroElement
import folium
import geopandas as gpd
import numpy.ma as ma
import rasterio
import rasterio.warp
import shapely.wkt
from osgeo import gdal
gdal.UseExceptions()

import ipywidgets as widgets
from ipywidgets import Layout

util_relative_from_notebook = os.path.abspath('../..')
util_relative_from_papermill_script = os.path.abspath('..')
sys.path.append(util_relative_from_notebook)
sys.path.append(util_relative_from_papermill_script)

from util.template import legend_template
import util.geo as util

## **1. Select the directory holding your OPERA RTC sample data**

Sample burst data may be retrieved and mosaiced with OPERA_RTC_download_reproject_mosaic_sample_bursts.ipynb

Note: local incidence angle maps, ellipsoidal incidence maps, layover-shadow masks, and dual-pol backscatter geotiffs are needed.

```
OPERA_L2-RTC_S1* ──
                  │
                  │─  OPERA_L2_RTC_VH_S1*.tif
                  │─  OPERA_L2_RTC_VV_S1*.tif 
                  │─  OPERA_L2_RTC_incidence_angle_S1*.tif
                  │─  OPERA_L2_RTC_local_incidence_angle_S1*.tif
                  │─  OPERA_L2_RTC_mask_S1*.tif

```

In [ ]:
print("Select the directory holding your data with the file structure shown above")
fc = FileChooser(Path.cwd(), layout=Layout(width='1000px'))
display(fc)

**Gather paths to data and create an output directory**

In [ ]:
# try/except for Papermill
try:
    data_dir = Path(fc.selected_path)
except:
    pass

In [ ]:
data_dir = Path(data_dir) # for Papermill
local_inc_angle = list(data_dir.glob('OPERA_L2_RTC-S1_local_incidence_angle_S1*.tif'))[0]
inc_angle = list(data_dir.glob('OPERA_L2_RTC-S1_incidence_angle_S1*.tif'))[0]
ls_mask = list(data_dir.glob('OPERA_L2_RTC-S1_mask_S1*.tif'))[0]
vh = list(data_dir.glob('OPERA_L2_RTC-S1_VH_S1*.tif'))[0]
vv = list(data_dir.glob('OPERA_L2_RTC-S1_VV_S1*.tif'))[0]

product_name = vh.parent.stem
output_dir = data_dir.parents[1]/f"intermediary_flattening_data/{product_name}_prepped_for_slope_comparison"
if not output_dir.is_dir():
    output_dir.mkdir()

## **2. Identify and Download Copernicus Global Land Cover Tiles**

**Create a GeoDataFrame containing input data and their geometries in their source CRS**

In [ ]:
ll_ur_corner_coords = [util.get_corner_coords(d)[0] + util.get_corner_coords(d)[1] 
                       for d in [vh, vv, local_inc_angle, inc_angle, ls_mask]]
geometry = [util.poly_from_minx_miny_maxx_maxy(c) for c in ll_ur_corner_coords]

gdf = gpd.GeoDataFrame(
    {
        "dataset": ['vh', 'vv', 'local_inc_angle', 'inc_angle', 'ls_mask'],
        "geometry": geometry
    }
)
gdf = gdf.set_geometry("geometry")
gdf = gdf.set_crs(f"epsg:{util.get_projection(vh)}")
gdf

**Create a GeoDataFrame in Lat/Long**

In [ ]:
gdf_4326 = gdf.to_crs(4326)
gdf_4326

**Find the set of tile strings for land cover tiles needed to cover AOI of sample OPERA data**

In [ ]:
vh_bounds_4326 = (
    gdf_4326.bounds.iloc[0].minx,
    gdf_4326.bounds.iloc[0].miny,
    gdf_4326.bounds.iloc[0].maxx,
    gdf_4326.bounds.iloc[0].maxy
)

lc_tile_intersections = util.landcover_100_tile_intersections(vh_bounds_4326)

lc_tile_str = [f"W{str(abs(c)).zfill(3)}" if c < 0 and i in [0, 2]
               else f"E{str(abs(c)).zfill(3)}" if c >= 0 and i in [0, 2] 
               else f"S{str(abs(c)).zfill(2)}" if c < 0 and i in [1, 3]
               else f"N{str(abs(c)).zfill(2)}" 
               for (i, c) in enumerate(lc_tile_intersections)]

if len(set(lc_tile_str)) == 2:
    land_cover_tile_str = [f"{lc_tile_str[0]}{lc_tile_str[1]}"]
elif len(set(lc_tile_str)) == 3:
    if lc_tile_str[0] == lc_tile_str[2]:
        land_cover_tile_str = [
            f"{lc_tile_str[0]}{lc_tile_str[1]}",
            f"{lc_tile_str[0]}{lc_tile_str[3]}",
    ]
    else:    
        land_cover_tile_str = [
            f"{lc_tile_str[0]}{lc_tile_str[1]}",
            f"{lc_tile_str[2]}{lc_tile_str[1]}",
        ]
elif len(set(lc_tile_str)) == 4:
    land_cover_tile_str = [
        f"{lc_tile_str[0]}{lc_tile_str[1]}",
        f"{lc_tile_str[0]}{lc_tile_str[3]}",
        f"{lc_tile_str[2]}{lc_tile_str[1]}",
        f"{lc_tile_str[2]}{lc_tile_str[3]}"
    ]
    
land_cover_tile_str

**Build the URLs to the land cover data**

In [ ]:
url = [(f"https://s3-eu-west-1.amazonaws.com/vito.landcover.global/v3.0.1/2019/{tile}/"
        f"{tile}_PROBAV_LC100_global_v3.0.1_2019-nrt_Discrete-Classification-map_EPSG-4326.tif") for tile in land_cover_tile_str
      ]
url

**Download land cover data and gather paths to them**

In [ ]:
for u in url:
    if not (data_dir/f"{u.split('/')[-1]}").exists():
        !wget -P {data_dir} {u}
        
land_cover = list(data_dir.glob('*PROBAV_LC100_global_v3.0.1_2019-nrt_Discrete-Classification-map_EPSG-4326.tif')) # change filename -> PROBAV_LC100
land_cover        

## **3. Merge Land Cover Data, If Needed**

In [ ]:
if len(land_cover) > 1:
    output_prefix = ''.join(l.name[0:8] for l in land_cover)

    merge_str = ''
    epsg = '4326'
    for tiff in land_cover:
        merge_str = f"{merge_str} {str(tiff)}"
    print(merge_str,'\n')

    gdal_cmd = f"gdal_merge.py -o {data_dir}/merged_{output_prefix}PROBAV_LC100_global_v3.0.1_2019-nrt_Discrete-Classification-map_EPSG-{epsg}.tif {merge_str}" # change filename -> PROBAV_LC100
    !$gdal_cmd
    
    land_cover = list(data_dir.glob('merged_*.tif'))[0]
else:
    land_cover = land_cover[0]

**Add mosaiced land cover data to lat/long GeoDataFrame**

In [ ]:
lc_corner_coords = util.get_corner_coords(land_cover)
lc_geometry = util.poly_from_minx_miny_maxx_maxy(lc_corner_coords[0]+lc_corner_coords[1])

gdf_4326.loc[len(gdf_4326)] = ['land_cover', lc_geometry]
gdf_4326

**Confirm that the land cover data spatially encompasses the OPERA Data**

In [ ]:
if all([gdf_4326.geometry.iloc[5].contains(gdf_4326.geometry.iloc[i]) for i in range(4)]):
    print("OPERA dataset bounds are contained by the landcover data 🎉\n")
else:
    raise Exception("One or more OPERA datasets are not contained by the bounds of the landcover data 😭😭😭")

location = [gdf_4326.bounds.iloc[0].maxy, gdf_4326.bounds.iloc[0].minx]
f = folium.Figure(width=1000, height=500)
m = folium.Map(location=location, zoom_start=4, tiles="CartoDB positron").add_to(f)

for i in [5, 0]:
    geo_series = gpd.GeoSeries(gdf_4326.geometry.iloc[i]).simplify(tolerance=0.001)
    geo_j = geo_series.to_json()
    fillColor = 'orange' if i == 0 else 'green'
    folium.features.GeoJson(data=geo_j, style_function=lambda x, fillColor=fillColor: {"fillColor": fillColor, 'fillOpacity': 0.5}).add_to(m)
    macro = MacroElement()
    macro._template = Template(legend_template(["#d2b934", "#8bbe86"], ["OPERA data bounds", "Landcover data bounds"]))
    m.get_root().add_child(macro)

m

## **4. Reproject Land Cover Data into EPSG of OPERA Data**

In [ ]:
resolution = 30
src_epsg = util.get_projection(land_cover)
dst_epsg = util.get_projection(vh)
reproj_path = land_cover.parent/f"{land_cover.stem.split('EPSG')[0]}EPSG-{dst_epsg}.tif"
reproj_path

In [ ]:
gdal.Warp(str(reproj_path), str(land_cover),
          srcSRS=f'EPSG:{src_epsg}', dstSRS=f'EPSG:{dst_epsg}',
          xRes=resolution, yRes=resolution, targetAlignedPixels=True,
         dstNodata=None, copyMetadata=True)


land_cover = reproj_path
land_cover

## **5. Create a Shapefile from the VH OPERA Data**

In [ ]:
a = gpd.GeoDataFrame(gdf.geometry.iloc[[0]])
a.to_file(data_dir/f"{''.join(vh.stem.split('vh_'))}_shape.shp")

In [ ]:
shp = list(data_dir.glob('*_shape.shp'))

to_clip = [land_cover, local_inc_angle, inc_angle, ls_mask, vh, vv]
to_clip

In [ ]:
for c in to_clip:
    print(str(c))

## **6. Clip All Rasters to Common Extents with Shapefile**

In [ ]:
for pth in tqdm(to_clip):
    clip = output_dir/f"{pth.stem}_clip.tif"
    if clip.is_file():
        clip.unlink()
    subset_command = f'gdalwarp -cutline {str(shp[0])} -crop_to_cutline {str(pth)} {str(clip)} -dstnodata {np.nan}'
    if pth == land_cover:
        land_cover = clip
    !$subset_command

## **7. Copy Shapefile into Output Directory**

In [ ]:
shp_pths = list(data_dir.glob('*_shape.*'))
shp_pths

for s in shp_pths:
    shutil.copy(s, output_dir/s.name)

*Prep OPERA RTC CalVal Slope Comparison Data - Version 1.0.0 - August 2023*